

# CNN Implementation

Importing libraries

In [ ]:
!pip install keras-attention


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from keras import layers
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPool1D
from nltk.tokenize import word_tokenize
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import SGD
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional, BatchNormalization,Convolution1D,MaxPooling1D, Reshape, GlobalAveragePooling1D
import h5py
from keras import callbacks
from sklearn import metrics

Upload Data

In [ ]:
from google.colab import files
dataset = files.upload()

Saving D1_all_shuffled.csv to D1_all_shuffled.csv


Loading Data

In [ ]:
df = pd.read_csv(r"D1_all_shuffled.csv")
X = df.iloc[:,df.columns!= 'label']
y = df.iloc[:,-1]

print(X)
print(y)

       digitsHostname  PresentAtrateHost  AvgWordLenHost  LongWordLenHost  \
0                   0                  0       10.500000               18   
1                   0                  0        5.000000                9   
2                   5                  0        8.333333               13   
3                   0                  0        5.000000                7   
4                   0                  0        7.666667               15   
...               ...                ...             ...              ...   
83852               0                  0        4.000000                5   
83853               0                  0        5.666667               11   
83854               0                  0        4.333333                7   
83855               2                  0        3.333333                4   
83856               0                  0        5.000000                9   

       HostNameLen  domainLen  digitsDomain  dotsHost  HypensHost  \
0     

In [ ]:
print(X.iloc[0])

digitsHostname        0.000000
PresentAtrateHost     0.000000
AvgWordLenHost       10.500000
LongWordLenHost      18.000000
HostNameLen          22.000000
                       ...    
]                     0.000000
-                     0.000000
_                     0.052632
.                     0.105263
~                     0.000000
Name: 0, Length: 93, dtype: float64


Defining Keras Models

In [ ]:
hidden_dims = 128
nb_filter = 64
filter_length = 5 
# embedding_vector_length = 128
pool_length = 4
lstm_output_size = 93




model = Sequential()
model.add(Embedding(93, 64, input_length=93))
model.add(Conv1D(64, kernel_size=3, input_shape = (93, 1), activation = 'relu'))
# model.add(Convolution1D(nb_filter=nb_filter,
#                         filter_length=filter_length,
#                         border_mode='valid',
#                         activation='relu',
#                         subsample_length=1))
model.add(MaxPooling1D(pool_size=pool_length))
model.add(LSTM(lstm_output_size))
model.add(Dense(1))
model.add(Activation('sigmoid'))
print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 93, 64)            5952      
                                                                 
 conv1d_6 (Conv1D)           (None, 91, 64)            12352     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 22, 64)           0         
 1D)                                                             
                                                                 
 lstm_11 (LSTM)              (None, 93)                58776     
                                                                 
 dense_6 (Dense)             (None, 1)                 94        
                                                                 
 activation_5 (Activation)   (None, 1)                 0         
                                                     

In [ ]:
for layer in model.layers:
    print(layer.output_shape)

(None, 93, 64)
(None, 91, 64)
(None, 22, 64)
(None, 93)
(None, 1)
(None, 1)


In [ ]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    (None, 93, 64)            5952      
                                                                 
 conv1d_6 (Conv1D)           (None, 91, 64)            12352     
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 22, 64)           0         
 1D)                                                             
                                                                 
 lstm_11 (LSTM)              (None, 93)                58776     
                                                                 
 dense_6 (Dense)             (None, 1)                 94        
                                                                 
 activation_5 (Activation)   (None, 1)                 0         
                                                     

In [ ]:
def evaluate_model(X_test,y_test):
    loss, acc = model.evaluate(X_test,y_test)
    return acc

In [ ]:
kf = KFold(n_splits=5)
accuracies = []

start = datetime.now()
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    print(X_train)
    y_train, y_test = y[train_index], y[test_index]
    y_train = y_train.values.reshape(-1,1)
    y_test  = y_test.values.reshape(-1,1)
    print(y_train)
    # model.fit(X_train,y_train,batch_size = 64,epochs=10)
    # # model.save("hybridmodel")
    # accuracy = evaluate_model(X_test,y_test)
    # history = model.fit()
    model.fit(X_train, y_train, batch_size = 64, epochs = 10,shuffle=True,validation_data=(X_test,y_test)) 
    #model.summary()
    scores =model.evaluate(X_test, y_test, batch_size=200, verbose=0)
    print(scores)
    # history.loss_plot('epoch')
    # print(accuracy)
    # accuracies.append(accuracy)
end = datetime.now()
td = (end - start).total_seconds()
print("Total Execution Time: ",td)

TRAIN: [16772 16773 16774 ... 83854 83855 83856] TEST: [    0     1     2 ... 16769 16770 16771]
       digitsHostname  PresentAtrateHost  AvgWordLenHost  LongWordLenHost  \
16772               0                  0        5.000000                7   
16773               0                  0        8.500000               14   
16774               0                  0        8.500000               15   
16775               0                  0        4.400000                7   
16776               0                  0        4.333333                6   
...               ...                ...             ...              ...   
83852               0                  0        4.000000                5   
83853               0                  0        5.666667               11   
83854               0                  0        4.333333                7   
83855               2                  0        3.333333                4   
83856               0                  0        5.000000